# **Parametric Models and Regressors** 

During this tutorial, we will be using a parametric model and linear parametrization of the dynamics.

To facilitate discussuion we will use the model of Kuka IIWA, however note that everything should works fine with floating body models as well. Let us install the descriptions package and import the Kuka IIWA urdf.

In [ ]:
%%capture
!pip3 install robot_descriptions

In [ ]:
from robot_descriptions import iiwa_description

## **Model and Inertial Parameters**


It is clear that the robot's behaviour is interconnected with it's inertial parameters, that must be determined for each particular robot in order. The dependency on such parameters are critical and should be carefully investigated. 

In the DARLi we are focusing on inertial parameters, namely:

$$
    \boldsymbol{\theta}=\left[mx_c, my_{c}, mz_{c}, m, I_{x x}, I_{y y}, I_{z z}, I_{x y}, I_{x z}, I_{z y}\right] \in \mathbb{R}^{10}
$$
Where $m,x_{c},y_{c},z_{c}, I_{(\cdot)}$ are mass, the coordinates of the centre of mass and the elements of the inertia tensor respectively.

This vector completely define the inertial parameters of each rigid body in robot, however one should keep in mind, that for some particular application it may be augmented with additional parameters to account for phenomenas not included in to rigid body model i.e. coefficents of friction force. 

Note that **we do not consider geometric** (kinematic) parameters, such as length of links in manipulators, location of actuators, and other dimension related quantities (here we treat them as known).

The DARLi supports the parametric version of the models through the `darli.parametric` module, it's include both conventional and functional wrappers, while support of pure pinnochio is still under developements. 

In [ ]:
from darli.backend import CasadiBackend
from darli.parametric import Functional

The parametric model then is defined in the very same way as the regular one by feeding the backend to certain model class, take for example the `Functional` class:

In [ ]:
model = Functional(CasadiBackend(iiwa_description.URDF_PATH))

For the model composet of several rigid bodies, the parameters defined as following stacked vector:

$$\boldsymbol{\theta} = \begin{bmatrix} \boldsymbol{\theta}_1 & \boldsymbol{\theta}_2 & \dots & \boldsymbol{\theta}_{nb} \end{bmatrix} \in \mathbb{R}^{10nb}$$ 

where $nb$ is number of rigid bodies in the model, and $np = 10nb$:

In [ ]:
nb = model.nbodies

The numerical value of stacked parameters stored in URDF are retrieved as:

In [ ]:
urdf_params = model.backend.base_parameters()

While theier symbolical representation ( `SX` for the `CasadiBackend`) are stored in:

In [ ]:
cas_params = model.parameters

## **Parametric Equations of Motion, Energy and Torque Regressors**

 ### **Parametric EoM**
 
 The dependency on inertial paremeters is critical and should be carefully investigated, fortunately for the robots composed of the rigid bodies one may deduce quite an eye-catching property - **linearity** to a parameters or so called regressor form of inverse dynamics:

 $$
    \mathbf{M}(\mathbf{q}, \boldsymbol{\theta})\dot{\mathbf{v}} +
    \mathbf{c}(\mathbf{q},\mathbf{v}, \boldsymbol{\theta}) +
    \mathbf{g}(\mathbf{q}, \boldsymbol{\theta}) 
 = \boldsymbol{\Phi}(\mathbf{q},\mathbf{v},\dot{\mathbf{v}}) \boldsymbol{\theta}
$$

where $\boldsymbol{\Phi} \in \mathbb{R}^{nq \times np}$ is regressor matrix (torque regressor):
 

In [ ]:
model.regressors.torque

One may also calculate all of the components of EoM:

In [ ]:
inertia = model.inertia
gravity_vector = model.gravity
coriolis = model.coriolis
bias_force = model.bias_force  # combined effect of gravity and coriolis

### **Forward and Inverse Dynamics**

The overall inverse dynamics then can be calculated as follows:

In [ ]:
model.inverse_dynamics

In [ ]:
model.forward_dynamics

As with regular models one can easily add different bodies, contacts, external forces and input selectros, for example:

In [ ]:
model.update_selector(passive_joints=range(3))
model.forward_dynamics

### **Parametric State Space**

In [ ]:
from darli.state_space import StateSpace

In [ ]:
state_space = StateSpace(model)

In [ ]:
state_space.derivative

In [ ]:
state_space.rollout(dt=1e-3, n_steps=10)